# How to use regioinvent

Note that running this entire notebook will take you around 2 to 3 hours.

To be able to use regioinvent, you will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5), easier is to get it through activity-browser: https://github.com/LCA-ActivityBrowser/activity-browser
- a brightway project within which there is an ecoinvent database with either the version 3.9.1/3.10 cutoff
- to download the trade database stored here: https://doi.org/10.5281/zenodo.11583814

In [1]:
import pandas as pd
import numpy as np
import sys
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append(r'C:\Users\matth\PycharmProjects\Regiopremise')
import regioinvent

The initialization of the Regioinvent class requires 3 arguments:
- the name of the brightway2 project where you store ecoinvent and where regioinvent will be added
- the name you gave your ecoinvent database within your brightway2 project
- the version of the ecoinvent database ('3.9.1' or '3.10')

In [2]:
ei_version = '3.10.1'
year = '2050'

In [3]:
if ei_version == '3.10.1':
    main_ei_version = '3.10'
else:
    main_ei_version = ei_version

In [4]:
regio = regioinvent.Regioinvent(
    bw_project_name=f'ecoinvent{ei_version}', 
    ecoinvent_database_name=f'ecoinvent_cutoff_{ei_version}_image_SSP2-Base_{year}', 
    ecoinvent_version=main_ei_version,
)

First step of regioinvent is to spatialize the original ecoinvent database. This entails multiple steps:
- Creating a new biosphere database for spatialized elementary flows (e.g., Ammonia, CA-QC)
- Spatializing the elementary flows used within the ecoinvent database, based on the location of the process itself
- Fixing water processes of ecoinvent creating imbalances (e.g., apple production in Chile calls irrigation, RoW -> imbalance)

In [5]:
regio.spatialize_my_ecoinvent()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Title: Writing activities to SQLite3 database:
  Started: 01/29/2025 13:52:04
  Finished: 01/29/2025 13:52:15
  Total time elapsed: 00:00:11
  CPU %: 93.70
  Memory %: 2.47


2025-01-29 14:06:50,692 - Regioinvent - INFO - Extracting ecoinvent to wurst...


Getting activity data


100%|██████████| 34810/34810 [00:00<00:00, 50496.84it/s]


Adding exchange data to activities


100%|██████████| 1002648/1002648 [01:29<00:00, 11154.12it/s]


Filling out exchange data


100%|██████████| 34810/34810 [00:07<00:00, 4687.41it/s]
2025-01-29 14:08:50,372 - Regioinvent - INFO - Fixing water processes in ecoinvent...
2025-01-29 14:10:09,530 - Regioinvent - INFO - Spatializing ecoinvent...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:38


Title: Writing activities to SQLite3 database:
  Started: 01/29/2025 14:11:02
  Finished: 01/29/2025 14:13:41
  Total time elapsed: 00:02:38
  CPU %: 93.80
  Memory %: 30.74


This creates two databases in your brightway2 project:
- "biosphere3_spatialized_flows", which contains all the newly created spatialized elementary flows
- "the-name-of-your-ecoinvent-database regionalized", which is a copy of the original ecoinvent but using spatialized elementary flows and boasting new processes to fix water modeling

Because elementary flows are now spatialize, you will need a specific LCIA method to operate with any flow (spatialized or not). The following function imports such methods. Currently available: "IW v2.1". Soon available: "EF v3.1" and "ReCiPe2016 v1.1 (E)".

In [6]:
regio.import_fully_regionalized_impact_method(lcia_method='IW v2.1')

2025-01-29 14:31:46,150 - Regioinvent - INFO - Importing the fully regionalized version of IMPACT World+ v2.1 for ecoinvent3.10.


If you want to go further in the regionalization, i.e., to create new national production processes and to rely on trade data to regionalize consumption markets of ecoinvent, you can run the next function. There are 3 arguments:
- _trade_database_path_ which is the path where you stored the trade database you downloaded from Zenodo: https://doi.org/10.5281/zenodo.11583814
- _regioinvent_database_name_ which is simply the name that the created database will take
- _cutoff_ which is the cut-off (between 0 and 1) beyond which countries will be aggreated as RoW. For more info on what cutoff does, see section "Selection of countries for regionalization" of the Methodology.md file.

In [7]:
regio.regionalize_ecoinvent_with_trade(
    trade_database_path=r'C:\Users\matth\PycharmProjects\Regiopremise\trade_data.db',
    regioinvent_database_name=f'regioinvent_cutoff_{ei_version}_image_SSP2-Base_{year}',
    cutoff=0.99,
)

2025-01-29 14:33:35,393 - Regioinvent - INFO - Extracting and formatting trade data...
2025-01-29 14:35:53,843 - Regioinvent - INFO - Regionalizing main inputs of traded products of ecoinvent...
100%|██████████| 1982/1982 [1:15:40<00:00,  2.29s/it]
2025-01-29 15:51:34,773 - Regioinvent - INFO - Creating consumption markets...
100%|██████████| 1982/1982 [54:14<00:00,  1.64s/it]  
2025-01-29 16:45:49,765 - Regioinvent - INFO - Performing second order regionalization...
2025-01-29 16:46:36,701 - Regioinvent - INFO - Regionalizing the elementary flows of the regioinvent database...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:08:52


Title: Writing activities to SQLite3 database:
  Started: 01/29/2025 16:47:49
  Finished: 01/29/2025 16:56:42
  Total time elapsed: 00:08:52
  CPU %: 92.00
  Memory %: 44.58


UnknownObject: Exchange between ('regioinvent_cutoff_3.10.1_image_SSP2-Base_2050', '418fa9438e6f4bb6bc8111e2359c1bcb') and ('regioinvent_cutoff_3.10.1_image_SSP2-Base_2050', '0d57d8bb7fc043469b881eb8be7a05ce') is invalid - one of these objects is unknown (i.e. doesn't exist as a process dataset)

This automatically wrote the regioinvent database in your brightway project. So you can now go on brightway2 or AB to perform your LCAs normally with regioinvent.

In [11]:
[i for i in regio.regioinvent_in_wurst if i['code'] == '0d57d8bb7fc043469b881eb8be7a05ce']

[{'classifications': [('ISIC rev.4 ecoinvent',
    '2394:Manufacture of cement, lime and plaster'),
   ('CPC',
    '37440: Portland cement, aluminous cement, slag cement and similar hydraulic cements, except in the form of clinkers')],
  'comment': 'This process represents the global export market for cement, CEM III/A. It can be used as a proxy for global production market but is not a global production market as it does not include domestic production data. The shares come from export data from the BACI database for the commodity 252390. Data from BACI is already in physical units. An average of the 5 last years of export trade available data is taken (in general from 2018 to 2022). Countries are taken until 99.0% of the global production amounts are covered. The rest of the data is aggregated in a RoW (Rest-of-the-World) region.',
  'location': 'GLO',
  'database': 'regioinvent_cutoff_3.10.1_image_SSP2-Base_2050',
  'code': '0d57d8bb7fc043469b881eb8be7a05ce',
  'name': 'export marke